In [1]:
import pandas as pd
import pprint
import plotly.express as px
import numpy as np
from sklearn.manifold import TSNE
from scipy.stats import ttest_ind_from_stats
import re

In [2]:
! pip install unidecode
! pip install python-Levenshtein

In [3]:
import unidecode
import Levenshtein as levenshtein

Data can be found in doc_covidstress

## Data Load

In [4]:
df_global_fr = pd.read_csv("/work/COVIDiSTRESS_global_survey_May_30_2020(final_cleaned_file).csv")
df_explain = pd.read_csv("/work/COVIDiSTRESS global survey May 30 2020 (numeric values).csv", skiprows=lambda x: x > 2)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (54,55,56,57,58,59,60) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df_explain.rename(columns={'$a':'a'})

,StartDate,EndDate,Status,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,DistributionChannel,UserLanguage,...,Expl_Coping_15,Expl_Coping_16,Expl_coping_txt,Expl_media_1,Expl_media_2,Expl_media_3,Expl_media_4,Expl_media_5,Expl_media_6,Final_open
0,Start Date,End Date,Response Type,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Distribution Channel,User Language,...,I have found the following helpful for reducin...,I have found the following helpful for reducin...,Other?,During the period where Corona Virus has impac...,During the period where Corona Virus has impac...,During the period where Corona Virus has impac...,During the period where Corona Virus has impac...,During the period where Corona Virus has impac...,During the period where Corona Virus has impac...,"Finally, if there is anything else in your lif..."
1,"{""ImportId"":""startDate"",""timeZone"":""America/De...","{""ImportId"":""endDate"",""timeZone"":""America/Denv...","{""ImportId"":""status""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""America...","{""ImportId"":""_recordId""}","{""ImportId"":""distributionChannel""}","{""ImportId"":""userLanguage""}",...,"{""ImportId"":""QID44_15""}","{""ImportId"":""QID44_16""}","{""ImportId"":""QID46_TEXT""}","{""ImportId"":""QID48_1""}","{""ImportId"":""QID48_2""}","{""ImportId"":""QID48_3""}","{""ImportId"":""QID48_4""}","{""ImportId"":""QID48_5""}","{""ImportId"":""QID48_6""}","{""ImportId"":""QID49_TEXT""}"


In [27]:
df_global_fr.shape

(9499, 130)

# Filter french data

In [7]:
df_global_fr = df_global_fr.loc[df_global_fr.Country=="France"].loc[df_global_fr.UserLanguage=="FR"].loc[df_global_fr.Dem_Expat!='yes']
df_global_fr.replace(["None"], [None], inplace=True)

In [8]:
df_global_fr.RecordedDate.min(), df_global_fr.RecordedDate.max()

('2020-03-30 11:50:46', '2020-05-27 15:31:09')

# Remove or infer nan value

## Data description
### Demographic
* Dem_dependents : Number of dependents (i.e. family members relying on you for support. Usually children)
* Dem_isolation_adults : number adults in same place
* Dem_isolation_kids : number children under the age of 12 are staying together in the same place as you are
* Dem_age
* Dem_gender
* Dem_edu
* Dem_edu_mom
* Dem_employment
* Dem_state
### Evaluative
* Scale_PSS10_UCLA_n (1 to 10): perceived stress for the past week
    * 5-Point Likert Scale (1=Never, 5= very often)
* Scale_PSS10_UCLA_n (11 to 13): perceived loneliness
    * 5-Point Likert Scale (1=Never, 5= very often)
* OECD_institutions_n : trust in country's government and health system 
    * 10-Point Likert Scale (0=Not at all, 10= completely)
* Corona_concerns_n : concerns ver coronavirus
    * 6-Point Likert Scale (1=strongly disagree,6= strongly agree)
* Compliance : overll compliance with local prevention guidelines
    * 6-Point Likert Scale (1=strongly disagree, 6= strongly agree)
* BFF_15_n : personnality
    * 6-Point Likert Scale (1=strongly disagree, 6= strongly agree)
* Expl_Distress_n : stressors and sources of distress
    * 6-Point Likert Scale (1=strongly disagree, 6= strongly agree, 7= does not apply to my current situation + free text "other")
* SPS_n : available social provisions in critical/distressing situations
    * 6-Point Likert Scale (1=strongly disagree, 6= strongly agree)
* Expl_Coping_n : Coping/decreasing discomfort
    * 6-Point Likert Scale (1=strongly disagree, 6= strongly agree) + free text "other"
* Expl_media_1 : Information sources and media behavior
    * 6-Point Likert Scale (1=strongly disagree, 6= strongly agree)

* Dem_riskgroup : close relations high-risk group for Coronavirus
    * 1=Yes, 2=No, 3=not sure
* Dem_islolation : situation
    * *Life carries on with minor changes* and *Isolated* and *Life carries on as usual* and *Isolated in medical facility of similar location*  

Deal with problematic field

In [9]:
df_global_fr.drop(columns=["AD_check"], inplace=True)
df_global_fr.drop(columns=["Duration..in.seconds."], inplace=True)
df_global_fr.drop(columns=["Unnamed: 0"], inplace=True)
df_global_fr.drop(columns=["PSS10_avg"], inplace=True)
df_global_fr.drop(columns=["SLON3_avg"], inplace=True)
df_global_fr.drop(columns=["neu"], inplace=True)
df_global_fr.drop(columns=["ext"], inplace=True)
df_global_fr.drop(columns=["ope"], inplace=True)
df_global_fr.drop(columns=["agr"], inplace=True)
df_global_fr.drop(columns=["con"], inplace=True)
df_global_fr.drop(columns=["SPS_avg"], inplace=True)
# Scale_SLON_2 not here

remove nan columns

In [10]:
print(df_global_fr.shape)
null_by_column = df_global_fr.isnull().sum(axis=0)
pprint.pprint({c: f"{cnull}/{df_global_fr.shape[0]}({cnull/df_global_fr.shape[0]*100:.2f}%)" for c, cnull in null_by_column.where(null_by_column>0).dropna().iteritems()})

(11966, 142)
{'AD_gain': '6268.0/11966(52.38%)',
 'AD_loss': '6504.0/11966(54.35%)',
 'BFF_15_1': '1463.0/11966(12.23%)',
 'BFF_15_10': '1459.0/11966(12.19%)',
 'BFF_15_11': '1456.0/11966(12.17%)',
 'BFF_15_12': '1455.0/11966(12.16%)',
 'BFF_15_13': '1455.0/11966(12.16%)',
 'BFF_15_14': '1449.0/11966(12.11%)',
 'BFF_15_15': '1448.0/11966(12.10%)',
 'BFF_15_2': '1452.0/11966(12.13%)',
 'BFF_15_3': '1450.0/11966(12.12%)',
 'BFF_15_4': '1455.0/11966(12.16%)',
 'BFF_15_5': '1458.0/11966(12.18%)',
 'BFF_15_6': '1454.0/11966(12.15%)',
 'BFF_15_7': '1457.0/11966(12.18%)',
 'BFF_15_8': '1470.0/11966(12.28%)',
 'BFF_15_9': '1459.0/11966(12.19%)',
 'Compliance_1': '1326.0/11966(11.08%)',
 'Compliance_2': '1342.0/11966(11.22%)',
 'Compliance_3': '1339.0/11966(11.19%)',
 'Compliance_4': '1343.0/11966(11.22%)',
 'Compliance_5': '1330.0/11966(11.11%)',
 'Compliance_6': '1321.0/11966(11.04%)',
 'Corona_concerns_1': '1200.0/11966(10.03%)',
 'Corona_concerns_2': '1224.0/11966(10.23%)',
 'Corona_concern

In [11]:
# Attributes lists 
# Preparation before aggregation

OECD_institutions = [s for s in df_global_fr.columns if "OECD_insititutions" in s]
scale_PSS10_UCLA = [s for s in df_global_fr.columns if "Scale_PSS10_UCLA" in s]
scale_SLON = [s for s in df_global_fr.columns if "Scale_SLON" in s]
corona_concerns = [s for s in df_global_fr.columns if "Corona_concerns" in s]
compliance = [s for s in df_global_fr.columns if "Compliance" in s]
BFF_15 = [s for s in df_global_fr.columns if "BFF_15" in s]
expl_Distress = [s for s in df_global_fr.columns if "Expl_Distress" in s and s != "Expl_Distress_txt" ]
SPS = [s for s in df_global_fr.columns if "SPS" in s]
expl_Coping = [s for s in df_global_fr.columns if "Expl_Coping" in s]
expl_media = [s for s in df_global_fr.columns if "Expl_media" in s]
dem_riskgroup = ['Dem_riskgroup']
dem_islolation = ['Dem_islolation']

df_explain.rename(columns={"Scale_PSS10_UCLA_11": "Scale_SLON_1", "Scale_PSS10_UCLA_12": "Scale_SLON_2", "Scale_PSS10_UCLA_13": "Scale_SLON_3"}, inplace=True)
# evaluation from 0 to 10
point_likert_scale_10 = OECD_institutions
# evualuation from 1 to 5
point_likert_scale_5 = scale_PSS10_UCLA + scale_SLON
# evaluation from 1 to 6
point_likert_scale_6 = corona_concerns + compliance + BFF_15 + expl_Distress + SPS + expl_Coping + expl_media
# evaluation Yes No Not sure
yes_no = dem_riskgroup 
# islolation ['Life carries on with minor changes', 'Isolated', 'Life carries on as usual', 'Isolated in medical facility of similar location']
islolation = dem_islolation

#Expl_Coping_n enlever les others
#Expl_Distress_n enlever others et 99
df_global_fr[expl_Distress] = df_global_fr[expl_Distress].replace(to_replace=[99], value=[None])

In [12]:
df_global_fr[point_likert_scale_10] = df_global_fr[point_likert_scale_10]/10
df_global_fr[point_likert_scale_5] = (df_global_fr[point_likert_scale_5]-1)/4
df_global_fr[point_likert_scale_6] = (df_global_fr[point_likert_scale_6]-1)/5

In [13]:
df_global_fr[point_likert_scale_6].describe()

,Corona_concerns_1,Corona_concerns_2,Corona_concerns_3,Corona_concerns_4,Corona_concerns_5,Compliance_1,Compliance_2,Compliance_3,Compliance_4,Compliance_5,...,Expl_Coping_13,Expl_Coping_14,Expl_Coping_15,Expl_Coping_16,Expl_media_1,Expl_media_2,Expl_media_3,Expl_media_4,Expl_media_5,Expl_media_6
count,10766.000000,10742.000000,10733.000000,10733.000000,10727.000000,10640.000000,10624.00000,10627.000000,10623.000000,10636.000000,...,9415.000000,9467.000000,9360.000000,9451.000000,9383.000000,9377.000000,9375.000000,9390.000000,9377.000000,9373.000000
mean,0.593219,0.798771,0.702823,0.776241,0.765396,0.796692,0.84070,0.866246,0.309837,0.659045,...,0.594116,0.761656,0.099316,0.589144,0.569711,0.549899,0.469440,0.460277,0.545932,0.362296
std,0.307048,0.218667,0.251756,0.232742,0.241127,0.209333,0.17755,0.169565,0.299149,0.247686,...,0.296118,0.221866,0.222909,0.267832,0.295722,0.307781,0.312371,0.293597,0.319193,0.273139
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.400000,0.800000,0.600000,0.600000,0.600000,0.800000,0.80000,0.800000,0.000000,0.600000,...,0.400000,0.600000,0.000000,0.400000,0.400000,0.200000,0.200000,0.200000,0.200000,0.200000
50%,0.600000,0.800000,0.800000,0.800000,0.800000,0.800000,0.80000,0.800000,0.200000,0.800000,...,0.600000,0.800000,0.000000,0.600000,0.600000,0.600000,0.600000,0.400000,0.600000,0.400000
75%,0.800000,1.000000,0.800000,1.000000,1.000000,1.000000,1.00000,1.000000,0.600000,0.800000,...,0.800000,1.000000,0.000000,0.800000,0.800000,0.800000,0.800000,0.600000,0.800000,0.600000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Drop columns with 25% of null or more

In [14]:
to_remove = null_by_column.where((null_by_column/df_global_fr.shape[0])>0.25).dropna()
print(to_remove.index.values)
df_global_fr.drop(columns=to_remove.index, inplace=True)

['AD_gain' 'AD_loss' 'born_92' 'experience_war' 'experience_war_TXT'
 'war_injury' 'loss_during_war' 'time_spent_in_war'
 'time_spent_in_war_TXT' 'Scale_UCLA_TRI_1' 'Scale_UCLA_TRI_2'
 'Scale_UCLA_TRI_3' 'Scale_UCLA_TRI_4' 'PS_PTSD_1' 'PS_PTSD_2' 'PS_PTSD_3'
 'PS_PTSD_4' 'PS_PTSD_5' 'Expl_Distress_txt' 'Expl_coping_txt'
 'Final_open' 'Scale_UCLA_TRI_avg']


Drop line with more than 25% of null

In [15]:
null_by_line = df_global_fr.isnull().sum(axis=1)
to_remove = null_by_line.where((null_by_line/df_global_fr.shape[1])>0.25).dropna()
print(len(to_remove.index), "lines to remove")
df_global_fr.drop(index=to_remove.index, inplace=True)

2467 lines to remove


region mapping

In [16]:
# Ne faîtes pas ça chez vous !!!!
['Île-de-France', 'Centre-Val de Loire', 'Bourgogne-Franche-Comté',
       'Normandie', 'Hauts-de-France', 'Grand-Est', 'Pays de la Loire',
       'Bretagne', 'Nouvelle-Aquitaine', 'Occitanie',
       'Auvergne-Rhône-Alpes', "Provence-Alpes-Côte d'Azur"]


def remove_accents(s):
    return unidecode.unidecode(s)

def format(x, min_lev_d=5):

    if x is None or x is np.nan:
        return None

    def prepross(x):
        x = re.sub(r"[\*\-\_/\(\)\.\']", " ", x)
        x = re.sub(r"\s+", " ", x)
        x = remove_accents(x)
        x = x.lower()
        return x

    def sub_leven(x, word, min_lev_d):
        return any(levenshtein.distance(xs, word)<=min_lev_d for xs in x.split(" "))
        
    x = prepross(x)

    region =['Île-de-France', 'Centre-Val de Loire', 'Bourgogne-Franche-Comté',
       'Normandie', 'Hauts-de-France', 'Grand-Est', 'Pays de la Loire',
       'Bretagne', 'Nouvelle-Aquitaine', 'Occitanie',
       'Auvergne-Rhône-Alpes', "Provence-Alpes-Côte d'Azur"]
    
    minlev = min([(levenshtein.distance(x, prepross(r)), r) for r in region], key=lambda x: x[0])


    if minlev[0] < min_lev_d:
        return minlev[1]

    if any(t in x for t in ["28", "50"]) or  sub_leven(x,"centre",3) or  sub_leven(x,"indre",1) or  sub_leven(x,"eure",1) or levenshtein.distance(x,"loire et cher")<2:
        return "Centre-Val de Loire"
    if "76" in x  or sub_leven(x,"normandie",1) or "normandie" in x:
        return "Normandie"
    if "22" in x or sub_leven(x,"bretagne",3) or sub_leven(x,"vilaine",2) or sub_leven(x,"finister",3):
        return "Bretagne"
    if sub_leven(x,"nord",1) or "hauts de france" in x or sub_leven(x,'picardie',2) or any(t in x for t in ["78", "59", "62"]):
        return 'Hauts-de-France'
    if sub_leven(x,"paris", 2) or "paris" in x or "le de france" in x  or sub_leven(x,"seine", 1)  or "marne" in x or any(t in x for t in ["77", "91", "92", "94", "95", "974"]):
        return 'Île-de-France'
    if sub_leven(x,"auvergne", 3) or any(t in x for t in ["01", "69", "63", "42", "74"]) or sub_leven(x,"rhones",2):
        return 'Auvergne-Rhône-Alpes'
    if sub_leven(x,"bourgogne",3) or "franche" in x or "21000" in x or "25" in x:
        return 'Bourgogne-Franche-Comté'
        #levenshtein.distance(x, "charente")<min_lev_d 
        # "charente" in x
    if sub_leven(x,"aquitaine",2) or sub_leven(x,"charente",2) or sub_leven(x,"deux",1) or "vienne" in x or any(t in x for t in ["17", "33", "86", "75"]):
        return 'Nouvelle-Aquitaine'
    if sub_leven(x, "reunion", 2):
        return "La Réunion"
    if sub_leven(x, "corse", 2):
        return "Corse"
    if '44' in x or '49' in x or sub_leven(x,"pays de la loire", 3) or sub_leven(x,"atlantique", 3) or "maine" in x:
        return 'Pays de la Loire'
    if sub_leven(x,"languedoc",3) or sub_leven(x,"midi",1) or sub_leven(x,"lozere",1) or sub_leven(x,'occitanie',3) or sub_leven(x,'vaucluse',2) or sub_leven(x,'ariege',1) or sub_leven(x,'pyrnes',1) or\
        any(t in x for t in ["66", "31", "34", "46", "47", "48"]) or\
        "garonne" in x or "rault" in x:
        return 'Occitanie'
    if sub_leven(x,"alsace",3) or sub_leven(x,"grand est",3) or sub_leven(x,"champagne",2) or\
        "grand est" in x or "gran est" in x or x == "est" or\
        any(t in x for t in ["51", "54", "57", "67", "68"]):
        return 'Grand-Est'
    if "06" in x or "13" in x or sub_leven(x,"paca",1) or sub_leven(x,"azur",1) or sub_leven(x,"maritime",1) or 'provence' in x:
        return "Provence-Alpes-Côte d'Azur"
    if sub_leven(x,"gualdeloupe", 2):
        return "Guadeloupe"
    if sub_leven(x,"caledonie", 2):
        return "Nouvelle Calédonie"
    if sub_leven(x,"guyane", 2):
        return "La Guyane"
    if x == " ":
        return None
    
    dico =  {
            'allier': "Auvergne-Rhône-Alpes",
            'angers': 'Pays de la Loire',
            'annecy': "Auvergne-Rhône-Alpes",
            'antilles': 'Les Antilles',
            'armorique': "Bretagne",
            'ath': None,
            'athe': None,
            'aube': "Grand-Est",
            'aude': "Grand-Est",
            'banlieue parisienne': 'Île-de-France',
            'bfc': 'Bourgogne-Franche-Comté' ,
            'bordeaux': 'Nouvelle-Aquitaine' ,
            'bourbonnais': "Auvergne-Rhône-Alpes",
            'catholique': None,
            'catholiques': None,
            'cherbourg': "Normandie",
            'vienne': 'Nouvelle-Aquitaine',
            'doubs': "Bourgogne-Franche-Comté",
            'drome': "Auvergne-Rhône-Alpes",
            'essonne': "Île-de-France",
            'etrchy': "Île-de-France",
            'faulquemont': "Grand-Est",
            'gap': "Provence-Alpes-Côte d'Azur",
            'garoult': "Provence-Alpes-Côte d'Azur",
            'gascogne': "Nouvelle-Aquitaine",
            'grand ouest': None ,
            'grenoble': "Auvergne-Rhône-Alpes",
            'hdf': "Hauts-de-France" ,
            'idf': 'Île-de-France' ,
            'ile de f': 'Île-de-France',
            'isere': "Auvergne-Rhône-Alpes",
            'la rochelle': 'Nouvelle-Aquitaine',
            'landes': "Nouvelle-Aquitaine",
            'le de 6': 'Bretagne',
            'les landes': "Nouvelle-Aquitaine",
            'lille': "Hauts-de-France" ,
            'limousin': 'Nouvelle-Aquitaine',
            'loireatlantique': 'Pays de la Loire',
            'marseille': "Provence-Alpes-Côte d'Azur",
            'martinique': 'La Martinique',
            'metz': "Grand-Est",
            'montauban': 'Occitanie' ,
            'montpellier': 'Occitanie' ,
            'montreuil': "Île-de-France",
            'morbihan': 'Bretagne' ,
            'moselle': "Grand-Est",
            'nancy': "Grand-Est",
            'normandie': 'Normandie',
            'var': "Provence-Alpes-Côte d'Azur",
            'ouest': None,
            'palaiseau': 'Île-de-France',
            'parisienne': 'Île-de-France',
            'perigord': 'Nouvelle-Aquitaine',
            'poitiers': 'Nouvelle-Aquitaine',
            'pologne': None,
            'polynsie': 'La Polynésie française',
            'puy de dme': "Auvergne-Rhône-Alpes",
            'ara': "Auvergne-Rhône-Alpes",
            'raa': "Auvergne-Rhône-Alpes",
            'aura' : "Auvergne-Rhône-Alpes",
            'rocheserviere': 'Pays de la Loire',
            'sarthes': 'Pays de la Loire',
            'savoie': "Auvergne-Rhône-Alpes",
            'seinesaintdenis': 'Île-de-France',
            'strasbourg': 'Grand-Est',
            'toulouse': 'Occitanie',
            'versailles': 'Île-de-France',
            'villeurbanne': 'Auvergne-Rhône-Alpes' ,
            'vosges': "Grand-Est",
            'yvelines': 'Île-de-France',
            'fr': None,
            'gard' : 'Occitanie',
            'jura' : 'Bourgogne-Franche-Comté',
            'lot' : 'Occitanie',
            'nivre':'Bourgogne-Franche-Comté',
            'nmes' : 'Occitanie',
            'pyrnes orientale' : 'Occitanie' ,
            'pyrnes orientales' : 'Occitanie',
            'ra' : 'Auvergne-Rhône-Alpes',
            'sans' : None,
            'sud' : 'Occitanie',
            'sud ouest' : 'Nouvelle-Aquitaine',
            'ain': 'Auvergne-Rhône-Alpes',
            'ain gex': 'Auvergne-Rhône-Alpes',
            'ardennes': "Grand-Est",
            'aucine': None ,
            'aucune': None,
            'aveyron': 'Occitanie',
            'bas rhin': "Grand-Est",
            'cannes la bocca': "Provence-Alpes-Côte d'Azur",
            'chelles': 'Île-de-France',
            'chrtienne': None,
            'clermont ferrand': 'Auvergne-Rhône-Alpes',
            'drme': 'Auvergne-Rhône-Alpes',
            'essone': 'Île-de-France' ,
            'evian les bains': 'Auvergne-Rhône-Alpes',
            'gers': 'Occitanie',
            'gironde': 'Nouvelle-Aquitaine',
            'haut doubs': 'Bourgogne-Franche-Comté',
            'haut rhin': "Grand-Est",
            'haute loire': 'Auvergne-Rhône-Alpes',
            'haute saone': 'Bourgogne-Franche-Comté',
            'haute savoie': 'Auvergne-Rhône-Alpes',
            'hautes alpes': "Provence-Alpes-Côte d'Azur",
            'hauts deseine': 'Île-de-France' ,
            'isre': 'Auvergne-Rhône-Alpes',
            'laudun': 'Occitanie',
            'les pavillons sous bois': 'Île-de-France',
            'loire': "Centre-Val de Loire",
            'loiret': "Centre-Val de Loire",
            'lorraine': "Grand-Est" ,
            'lyon': 'Auvergne-Rhône-Alpes',
            'mayenne': "Pays de la Loire",
            'meurthe et moselle': "Grand-Est",
            'nantes': 'Pays de la Loire',
            'nice': "Provence-Alpes-Côte d'Azur",
            'no': None,
            'non':  None,
            'non croyant': None,
            'puy de dome': "Auvergne-Rhône-Alpes",
            'rgion sud': 'Occitanie',
            'roissy en brie': "Île-de-France",
            'rouen': "Île-de-France",
            'rueil malmaison': "Île-de-France",
            'saint lieux les lavaur': "Occitanie",
            'saint pierre et miquelon': "Saint-Pierre et Miquelon",
            'saone et loire': "Bourgogne-Franche-Comté",
            'sarthe': "Pays de la Loire",
            'somme': "Hauts-de-France",
            'somme 80 ': "Hauts-de-France",
            'sone et loire': "Bourgogne-Franche-Comté",
            'st fort sur le n': "Nouvelle-Aquitaine",
            'tarn': "Occitanie",
            'tours': "Centre-Val de Loire",
            'val de loire': "Centre-Val de Loire",
            'vertou': "Loire-Atlantique",
            'voiron': "Auvergne-Rhône-Alpes",
            'yonne': 'Bourgogne-Franche-Comté',
            'sud est de la france' : "Grand-Est",
            'ardche' : 'Auvergne-Rhône-Alpes',
            'caen france': "Normandie",
            'france': None,
            'france aveyron' : 'Occitanie',
            'france isre' : "Auvergne-Rhône-Alpes",
            'loire, france' : "Pays de la Loire",
            'manche' : "Normandie",
            'rgion grans sud france':'Occitanie',
            "les antilles": "Les Antilles",
            "la martinique": "La Martinique",
            "la polynesie francaise": "La Polynesie francaise",
            'sud france':'Occitanie'}
    return dico[x] if x in dico else x

df_global_fr['Dem_state'] = df_global_fr['Dem_state'].apply(lambda x: format(x))
len(sorted(list(df_global_fr['Dem_state'].value_counts().index))), sorted(list(df_global_fr['Dem_state'].value_counts().index))

(22,
 ['Auvergne-Rhône-Alpes',
  'Bourgogne-Franche-Comté',
  'Bretagne',
  'Centre-Val de Loire',
  'Corse',
  'Grand-Est',
  'Guadeloupe',
  'Hauts-de-France',
  'La Guyane',
  'La Martinique',
  'La Polynésie française',
  'La Réunion',
  'Les Antilles',
  'Loire-Atlantique',
  'Normandie',
  'Nouvelle Calédonie',
  'Nouvelle-Aquitaine',
  'Occitanie',
  'Pays de la Loire',
  "Provence-Alpes-Côte d'Azur",
  'Saint-Pierre et Miquelon',
  'Île-de-France'])

Infer remaining null value

In [17]:
for c in df_global_fr.columns:
    bool_series = df_global_fr[c].isnull()
    nan_value = df_global_fr[c][bool_series]
    if nan_value.shape[0] > 0:
        true_value = df_global_fr[c][~bool_series]
        df_global_fr[c].loc[nan_value.index] = [true_value.sample(1).values[0] for i in range(nan_value.shape[0])]
df_global_fr.isnull().sum().sum()

/shared-libs/python3.7/py/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


0

In [18]:
df_global_fr["OECD_institutions_avg"] = df_global_fr[OECD_institutions].mean(axis=1)
df_global_fr["Scale_PSS10_UCLA_avg"] = df_global_fr[scale_PSS10_UCLA].mean(axis=1)
df_global_fr["Scale_SLON_avg"] = df_global_fr[scale_SLON].mean(axis=1)
df_global_fr["Corona_concerns_avg"] = df_global_fr[corona_concerns].mean(axis=1)
df_global_fr["Compliance_avg"] = df_global_fr[compliance].mean(axis=1)
df_global_fr["BFF_15_avg"] = df_global_fr[BFF_15].mean(axis=1)
df_global_fr["expl_Distress15_avg"] = df_global_fr[expl_Distress].mean(axis=1)
df_global_fr["SPS_avg"] = df_global_fr[SPS].mean(axis=1)
df_global_fr["expl_Coping_avg"] = df_global_fr[expl_Coping].mean(axis=1)
df_global_fr["expl_media_avg"] = df_global_fr[expl_media].mean(axis=1)

In [19]:
df_global_fr.shape, df_global_fr.isnull().sum().sum(), df_global_fr.isna().sum().sum()

((9499, 130), 0, 0)

In [20]:
df_global_fr.RecordedDate =  pd.to_datetime(df_global_fr.RecordedDate)

In [21]:
df_global_fr.to_csv("COVIDiSTRESS_cleaned.csv")
df_explain.to_csv("COVIDiSTRESS_explained.csv")

In [23]:
df_global_fr.columns

Index(['RecordedDate', 'UserLanguage', 'Dem_age', 'Dem_gender', 'Dem_edu',
       'Dem_edu_mom', 'Dem_employment', 'Country', 'Dem_Expat', 'Dem_state',
       ...
       'OECD_institutions_avg', 'Scale_PSS10_UCLA_avg', 'Scale_SLON_avg',
       'Corona_concerns_avg', 'Compliance_avg', 'BFF_15_avg',
       'expl_Distress15_avg', 'SPS_avg', 'expl_Coping_avg', 'expl_media_avg'],
      dtype='object', length=130)

In [53]:
df_global_fr_final = df_global_fr[df_global_fr.Dem_state.isin(['Île-de-France', 'Centre-Val de Loire', 'Bourgogne-Franche-Comté',
                                                               'Normandie', 'Hauts-de-France', 'Grand-Est', 'Pays de la Loire',
                                                               'Bretagne', 'Nouvelle-Aquitaine', 'Occitanie',
                                                               'Auvergne-Rhône-Alpes', "Provence-Alpes-Côte d'Azur"])]
df_global_fr_final_gr = df_global_fr_final.groupby(['Dem_state', pd.Grouper(key='RecordedDate', freq='W-MON')])["Scale_SLON_avg", "Scale_PSS10_UCLA_avg"].mean()
df_global_fr_final_gr["support"] = df_global_fr_final.groupby(['Dem_state', pd.Grouper(key='RecordedDate', freq='W-MON')])["Scale_SLON_avg"].count()
df_global_fr_final_gr.rename(columns={"Scale_SLON_avg": "loneliness_target_covidistress",
                                   "Scale_PSS10_UCLA_avg": "stress_target_covidistress"}, inplace=True)
df_global_fr_final_gr.index.rename(["region", "date"],
                                    inplace=True)
df_global_fr_final_gr

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """


loneliness_target_covidistress  \
region               date                                         
Auvergne-Rhône-Alpes 2020-04-06                        0.341838   
                     2020-04-13                        0.348825   
                     2020-04-20                        0.428161   
                     2020-04-27                        0.520833   
                     2020-05-04                        0.437500   
...                                                         ...   
Île-de-France        2020-05-04                        0.333333   
                     2020-05-11                        0.365000   
                     2020-05-18                        0.324074   
                     2020-05-25                        0.194444   
                     2020-06-01                        0.583333   

                                 stress_target_covidistress  support  
region               date                                             
Auvergne-Rhône-Alpes 2020-04-06                    0.479171     1019  
                     2020-04-13                    0.492308      156  
                     2020-04-20                    0.500862       29  
                     2020-04-27                    0.568750        4  
                     2020-05-04                    0.510417       12  
...                                                     ...      ...  
Île-de-France        2020-05-04                    0.493750       16  
                     2020-05-11                    0.470833      150  
                     2020-05-18                    0.525000        9  
                     2020-05-25                    0.558333        3  
                     2020-06-01                    0.425000        1  

[89 rows x 3 columns]

In [55]:
df_global_fr_final_gr["support"][df_global_fr_final_gr.support > 10].count()

45

In [48]:
df_global_fr_final_gr.to_csv("COVIDiSTRESS_region_date_FINAL.csv")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ee440d56-15d8-47e2-9125-c7817d63a1b6' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>